<a href="https://colab.research.google.com/github/JayrajRadadiya/Capstone-Project/blob/main/PythonCodes/SES_Survey_1_DataPush.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone - Push data Chunk
### 1st Survey

## Importing & Authenticating

In [ ]:
#Authenticating google colab
import google.colab
google.colab.auth.authenticate_user()

In [ ]:
from google.colab import files

# upload CSV file from local machine
uploaded = files.upload()

Saving Studentsurvey.csv to Studentsurvey.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install PyDrive library
!pip install PyDrive

In [ ]:
!pip install pycountry
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 85.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681847 sha256=241e9108c7097d94051114ec23373bcd43f6eb2e97fa0b81046feadae3dfd410
  Stored in directory: /root/.cache/pip/wheels/47/15/92/e6dc85fcb0686c82e1edbcfdf80cfe4808c058813fed0baa8f
Successfully built pycountry
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing Libraries
import io
import pandas as pd
import numpy as np
import pycountry
from fuzzywuzzy import fuzz
import re
from google.cloud import bigquery
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Creating a DataFrame

In [ ]:
#Creating a DataFrame Object - df and renaming it columns to a particular format , datatypes are corrected and the unwanted columns are dropped
df = pd.read_csv(io.BytesIO(uploaded['Studentsurvey.csv']))
df = df.rename(columns={
    'Current Semester': 'Semester',
    'Country of Origin': 'Country',
    'State of Origin': 'State',
    'Age Group': 'Age',
    'Highest level of Education': 'Education_Lvl',
    'Study Major': 'Majors',
    'Previous Experience': 'Work_Exp',
    'How much Experience': 'Yrs_Exp',
    'Industry ': 'Field_of_Exp',
    'Zipcode': 'Zip_code',
    'Accommodation ': 'Accm_Type',
    'Accommodation rating': 'Accm_finding_score',
    'Quality of Accom': 'Accm_Quality_Score',
    'Affordability rating': 'Accm_affordibility_score',
    'Commute rating': 'Commute_score',
    'Daily needs rating': 'Needs_Availibility_Score',
    'Exploring Platforms': 'Accm_info_source',
    'Current Emp Status': 'employment_status',
    'Timetable rating': 'Timetable_Schedules_score',
    'Section shuffling rating': 'Section_Allocation_score',
    'Weekly hours rating': 'Class_Hours_Score',
    'Academic resources rating': 'Course_Contents_Score',
    'Academic Workload rating': 'Afterclass_Workload_score',
    'Idea of Services': 'Services_Awareness',
    'Student services': 'Services_Used',
    'feedback': 'Feedback'
})
df = df.drop(columns=['Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32'])
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,Delhi,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Banking and Financial services,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,NaN,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,Karnataka,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,3,4,4,4,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,Punjab,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


## Data Cleaning

In [ ]:
#State (Text Field in Sruvey) - Using Fuzzywuzzy, validating the entered state with a valid state and also formatting it to desirable format
def is_valid_state(state):
    state_upper = state.upper()
    for subdivision in pycountry.subdivisions:
        subdivision_upper = subdivision.name.upper()
        score = fuzz.ratio(state_upper, subdivision_upper)
        if score >= 80: 
            return subdivision_upper
    return float('nan')
df['State'] = df['State'].apply(lambda x: is_valid_state(x))
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Banking and Financial services,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,NaN,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,3,4,4,4,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
#Field of Experience (Text Field in Survey) - Filling the missing values with No Experience and that grouping it and storing it in a new column

df['Field_of_Exp'] = df['Field_of_Exp'].fillna('No Experience')

def group_titles(title):
    if 'No Experience' in title:
        return 'No Experience'
    elif 'IT' in title or 'Information Technology' in title or 'Software' in title or 'it' in title:
        return 'Information Technology'
    elif 'Banking' in title or 'Finance' in title or 'Accounting' in title or 'FinTech' in title:
        return 'Finance & Banking'
    elif 'Data' in title or 'Analytics' in title or 'Statistician' in title or 'data' in title or 'Business Data Analyst' in title:
        return 'Data Analytics'
    elif 'Marketing' in title or 'Public Relations' in title or 'Business Development' in title or 'Sales' in title or 'Account Manager' in title or 'Digital Marketing' in title or 'Content Marketing' in title or 'Marketing Analytics' in title:
        return 'Sales & Marketing'
    elif 'Manufacturing' in title or 'Operations' in title or 'Supply Chain' in title:
        return 'Manufacturing'
    elif 'Healthcare' in title or 'Medical' in title or 'Pharmaceutical' in title or 'Health Insurance' in title:
        return 'Healthcare'
    elif 'Media' in title or 'Entertainment' in title or 'Broadcasting' in title:
        return 'Media/Entertainment'
    elif 'Education' in title or 'Teaching' in title or 'Research' in title:
        return 'Education'
    elif 'Engineering' in title or 'Mechanical' in title or 'Aeronautical' in title or 'Architecture' in title or 'Electronics' in title or 'Electrical' in title or 'Communication' in title or 'Telecom' in title or 'Automotive' in title:
        return 'Engineering'
    elif 'Customer Service' in title or 'Hospitality' in title or 'Client Services' in title or 'Retail' in title or 'Merchandising' in title:
        return 'Customer Service'
    elif 'Project Management' in title or 'Operations Management' in title or 'Program Management' in title:
        return 'Project Management'
    elif 'Human Resources' in title or 'Recruiting' in title:
        return 'Human Resources'
    elif 'Insurance' in title:
        return 'Insurance'
    elif 'Government' in title or 'Military' in title:
        return 'Government'
    elif 'BPO' in title or 'KPO' in title or 'RPO' in title:
        return 'BPO/KPO/RPO'
    else:
        return 'Other'

        
df['Field_of_Exp'] = df['Field_of_Exp'].astype(str)
df['Field_of_Exp'] = df['Field_of_Exp'].apply(group_titles)

df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,NaN,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,3,4,4,4,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
#Zipcode - Formatting Zipcode in a desirable format

pattern = re.compile('[A-Z0-9]')
#col_to_format = 'Zip_code'

def format_zip(zip_code):
    if isinstance(zip_code, str) and re.match('\s*[A-Z0-9]{6}\s*', zip_code):
        alphanumeric_chars = pattern.findall(zip_code)
        padded_string = ''.join(alphanumeric_chars)[:6].ljust(6)
        formatted_string = padded_string[:3] + ' ' + padded_string[3:]
        return formatted_string
    else:
        return zip_code

df['Zip_code'] = df['Zip_code'].apply(format_zip)
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,NaN,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A 5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,3,4,4,4,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,NaN,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ID                         300 non-null    int64 
 1   Intake                     300 non-null    object
 2   Semester                   300 non-null    object
 3   Country                    300 non-null    object
 4   State                      257 non-null    object
 5   Age                        300 non-null    object
 6   Gender                     300 non-null    object
 7   Education_Lvl              300 non-null    object
 8   Majors                     300 non-null    object
 9   Work_Exp                   300 non-null    object
 10  Yrs_Exp                    199 non-null    object
 11  Field_of_Exp               300 non-null    object
 12  Zip_code                   300 non-null    object
 13  Accm_Type                  300 non-null    object
 14  Accm_findi

In [ ]:
# Filling all the missing values in below columns with 'None'

df['Services_Used']=df['Services_Used'].replace(np.nan,'None')
df['Feedback']=df['Feedback'].replace(np.nan,'None')
df['Accm_info_source']=df['Accm_info_source'].replace(np.nan,'None')
df['Yrs_Exp']=df['Yrs_Exp'].replace(np.nan,'None')
df['State']=df['State'].replace(np.nan,'None')
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,None,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A 5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,None,Part-time,3,4,4,4,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,None,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


## Big-Query Table
### API Connection to Google BigQuery Database (surveyproject-378222.Capstone_Project)

In [ ]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.Base_Survey_Table`;
"""
query_job = client.query(sc)
query_job.result()
table_id = 'surveyproject-378222.Capstone_Project.Base_Survey_Table'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Intake', 'STRING'),
    bigquery.SchemaField('Semester', 'STRING'),
    bigquery.SchemaField('Country', 'STRING'),
    bigquery.SchemaField('State', 'STRING'),
    bigquery.SchemaField('Age', 'STRING'),
    bigquery.SchemaField('Gender', 'STRING'),
    bigquery.SchemaField('Education_Lvl', 'STRING'),
    bigquery.SchemaField('Majors', 'STRING'),
    bigquery.SchemaField('Work_Exp', 'STRING'),
    bigquery.SchemaField('Yrs_Exp', 'STRING'),
    bigquery.SchemaField('Field_of_Exp', 'STRING'),
    bigquery.SchemaField('Zip_code', 'STRING'),
    bigquery.SchemaField('Accm_Type', 'STRING'),
    bigquery.SchemaField('Accm_finding_score', 'STRING'),
    bigquery.SchemaField('Accm_Quality_Score', 'STRING'),
    bigquery.SchemaField('Accm_affordibility_score', 'STRING'),
    bigquery.SchemaField('Commute_score', 'STRING'),
    bigquery.SchemaField('Needs_Availibility_Score', 'STRING'),
    bigquery.SchemaField('Accm_info_source', 'STRING'),
    bigquery.SchemaField('employment_status', 'STRING'),
    bigquery.SchemaField('Timetable_Schedules_score', 'INTEGER'),
    bigquery.SchemaField('Section_Allocation_score', 'INTEGER'),
    bigquery.SchemaField('Class_Hours_Score', 'INTEGER'),
    bigquery.SchemaField('Course_Contents_Score', 'INTEGER'),
    bigquery.SchemaField('Afterclass_Workload_score', 'INTEGER'),
    bigquery.SchemaField('Services_Awareness', 'STRING'),
    bigquery.SchemaField('Services_Used', 'STRING'),
    bigquery.SchemaField('Feedback', 'STRING')
]

try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

#Inserting Data into to Big-Query Base Table
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()

Table Base_Survey_Table created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=922c00d2-eb72-47d6-871c-e79422953aab>

## Breaking Base_Survey_Table to 7 other Tables

In [ ]:
 #Defining the SQL code to create tables and insert data
sql_code = """
DROP TABLE IF EXISTS `Capstone_Project.Academic`;
CREATE TABLE `Capstone_Project.Academic` (
  ID INT NOT NULL,
  intake STRING,
  semester STRING
);
INSERT INTO `Capstone_Project.Academic`
SELECT ID, intake, semester 
FROM `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Academic_feedback`;
CREATE TABLE `Capstone_Project.Academic_feedback` (
  ID INT NOT NULL,
  Timetable_Schedules_score INT,
  Section_Allocation_score INT,
  Class_Hours_Score INT,
  Course_Contents_Score INT,
  Afterclass_Workload_score INT
);
INSERT INTO `Capstone_Project.Academic_feedback`
SELECT ID, Timetable_Schedules_score, Section_Allocation_score, Class_Hours_Score, Course_Contents_Score, Afterclass_Workload_score 
FROM `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Accommodation`;
CREATE TABLE `Capstone_Project.Accommodation` (
  ID INT NOT NULL,
  Zip_code STRING,
  Accm_Type STRING,
  Accm_info_source STRING
);
INSERT INTO `Capstone_Project.Accommodation`
SELECT ID, Zip_code, Accm_Type, Accm_info_source
FROM `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Accommodation_feedback`;
CREATE TABLE `Capstone_Project.Accommodation_feedback` 
(
  ID INT NOT NULL,
  Accm_finding_score STRING,
  Accm_Quality_Score STRING,
  Accm_affordibility_score STRING,
  Commute_score STRING,
  Needs_Availibility_Score STRING
);

 insert into `surveyproject-378222.Capstone_Project.Accommodation_feedback`
select 
  ID,
Accm_finding_score ,
  Accm_Quality_Score ,
  Accm_affordibility_score ,
  Commute_score ,
  Needs_Availibility_Score 
from `surveyproject-378222.Capstone_Project.Base_Survey_Table` ;

DROP TABLE IF EXISTS `Capstone_Project.Demographics`;
CREATE TABLE `Capstone_Project.Demographics` 
(
  ID INT NOT NULL,
  Country STRING,
  State STRING ,
  Age STRING,
  Gender STRING,
  Education_lvl STRING,
  Majors STRING,
  Work_exp STRING,
  Yrs_exp STRING,
  Field_of_exp STRING,
  employment_status STRING,
);

insert into `surveyproject-378222.Capstone_Project.Demographics`
select 
  ID,
  Country ,
  State ,
  Age ,
  Gender ,
  Education_lvl ,
  Majors ,
  Work_exp ,
  Yrs_exp ,
  Field_of_exp ,
  employment_status
from `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Feedback`;
CREATE TABLE `Capstone_Project.Feedback` (
  ID INT NOT NULL,
  Feedback STRING
);

INSERT INTO `Capstone_Project.Feedback` (ID, Feedback)
SELECT ID, Feedback FROM `Capstone_Project.Base_Survey_Table`;


DROP TABLE IF EXISTS `Capstone_Project.Services`;
CREATE TABLE `Capstone_Project.Services` 
(
  ID INT NOT NULL,
  Services_Awareness STRING,
  Services_Used STRING
);

insert into `surveyproject-378222.Capstone_Project.Services`
select 
  ID,
  Services_Awareness,
  Services_Used
from `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

"""
query_job = client.query(sql_code)
query_job.result()
